主线程，子线程，主线程分别是什么？

A启动B，那么A是B的父线程，B是A的子线程； Python启动的第一个线程叫主线程！

来到ipython环境来，ipython启动的就是主线程；主线程就是爷爷线程！

并发：假同时  一段时间内同时处理多个任务  单核也可以并发

并行：真同时  同时处理多个任务   必须要多核

Python中实现并发的手段有哪些？

主流操作系统：
* 线程
* 进程 

主流的语言通常提供用户空间的调度：协程

## 线程

事实上，Linux的线程是通过进程来实现的！

Python里面，进程和线程有所区别！

Python里，每个进程都会启动一个解释器；线程会共享一个解释器。

Python的线程是通过标准库来实现的！  threading

In [1]:
import threading

In [2]:
def worker():
    print('work')

In [3]:
thread = threading.Thread(target=worker)  ## 创建线程对象 target参数是一个函数，这个函数即线程要执行的逻辑

In [4]:
thread.start()   ## start 方法启动一个线程， 当这个线程的逻辑执行完毕的时候 ，线程自动退出，Pyhton没有提供主动退出线程的方法

work


所以，一定要注意线程的退出

In [18]:
import time
def worker(num):
    time.sleep(1)
    print('worker-{}'.format(num))

In [19]:
for x in range(5):
    t = threading.Thread(target=worker, args=(x, ))  ## 这里args是函数(这里为worker)传递参数的！
    t.start()                    ##  启动             ## 发现最后是一瞬间出来的，并没有sleep

worker-0
worker-1
worker-3
worker-2
worker-4


如何标识一个线程

In [20]:
threading.current_thread()             ##  返回当前线程

<_MainThread(MainThread, started 140438176315200)>

In [21]:
threading.Thread(target=lambda:print(threading.current_thread())).start()

<Thread(Thread-26, started 140437469767424)>


In [22]:
thread = threading.current_thread()

In [23]:
thread.name       ##  获得当前线程的名字

'MainThread'

In [24]:
thread.ident    ## 得到当前一个线程号

140438176315200

In [25]:
thread.is_alive()   ## 这是一个方法，加括号了 ，表示这个线程是否还活着

True

如何显示的更好看呢？

### logging

In [ ]:
## 可以看标准库的用法，这里只简单讲讲

In [27]:
import logging

In [35]:
logging.warn('haha')

/root/.pyenv/versions/3.5.2/envs/python/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  if __name__ == '__main__':


提示warn已经被warning替代了

In [36]:
logging.warning('hahah')

In [37]:
logging.error('hahah')

In [38]:
logging.info('hahah')

这里跟讲的不一样，讲的里只有info没有输出      可能跟ipython环境有关，使用pycharm确实是可以的！

In [39]:
logging.basicConfig(level=logging.DEBUG)

In [40]:
logging.info('haha')   ## 这里也是，pycharm中就可以！

In [41]:
help(logging.basicConfig)

Help on function basicConfig in module logging:

basicConfig(**kwargs)
    Do basic configuration for the logging system.
    
    This function does nothing if the root logger already has handlers
    configured. It is a convenience method intended for use by simple scripts
    to do one-shot configuration of the logging package.
    
    The default behaviour is to create a StreamHandler which writes to
    sys.stderr, set a formatter using the BASIC_FORMAT format string, and
    add the handler to the root logger.
    
    A number of optional keyword arguments may be specified, which can alter
    the default behaviour.
    
    filename  Specifies that a FileHandler be created, using the specified
              filename, rather than a StreamHandler.
    filemode  Specifies the mode to open the file, if filename is specified
              (if filemode is unspecified, it defaults to 'a').
    format    Use the specified format string for the handler.
    datefmt   Use the specified 

In [44]:
def worker(num):
    logging.warning('worker-{}'.format(num))   ##  用loggin替代print，输出的时候就归整了！

In [45]:
for x in range(5):
    t = threading.Thread(target=worker, args=(x, ))  ## 这里args是函数(worker)传递参数的！
    t.start()     

In [ ]:
## 但是并没有输出，估计还是ipython的事!  pycharm测试，可以归整显示！

所以，通常会用logging来替代print的！

**logging是线程安全的，print是线程不安全的；记住这点就可以啦！**

记住，**在多线程环境中**，用到print的时候，统统用logging来代替！ 记住这句话！！

logging就是print的替代品，可以做日志功能。

线程安全是比较大的一个主题，后面会有

In [2]:
import logging
import importlib

importlib.reload(logging)   ##  reload 就是重新导入的意思    
                            ##  只有在ipython里才需要reload 在IDE里面是不需要reload的

<module 'logging' from '/root/.pyenv/versions/3.5.2/lib/python3.5/logging/__init__.py'>

In [3]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

In [4]:
logging.warning('haha')

2017-04-14 14:21:58,986 WARNING  [MainThread] haha


In [5]:
logging.info('xixi')

2017-04-14 14:22:34,757 INFO  [MainThread] xixi


In [6]:
logging.debug('haha')

2017-04-14 14:22:43,397 DEBUG  [MainThread] haha


In [7]:
def worker(num):
    logging.warning('worker-{}'.format(num))

In [10]:
for x in range(5):
    t = threading.Thread(target=worker, args=(x,))
    t.start()

2017-04-14 14:25:10,702 WARNING  [Thread-4] worker-0
2017-04-14 14:25:10,704 WARNING  [Thread-5] worker-1
2017-04-14 14:25:10,705 WARNING  [Thread-6] worker-2
2017-04-14 14:25:10,715 WARNING  [Thread-7] worker-3
2017-04-14 14:25:10,718 WARNING  [Thread-8] worker-4


### 参数

In [11]:
def add(x,y):
    logging.info(x + y)

In [13]:
add(1,2)

2017-04-14 15:21:26,503 INFO  [MainThread] 3


In [16]:
add(x=2, y=6)

2017-04-14 15:21:51,466 INFO  [MainThread] 8


In [17]:
threading.Thread(target=add, args=(1,2)).start()

2017-04-14 15:22:38,536 INFO  [Thread-9] 3


In [18]:
threading.Thread(target=add, kwargs={'x':1, 'y':2}).start()

2017-04-14 15:23:50,653 INFO  [Thread-10] 3


In [19]:
threading.Thread(target=add, args=(1,),kwargs={'y':2}).start()  ## kwargs以字典的方式传递参数

2017-04-14 15:25:07,663 INFO  [Thread-11] 3


通过args参数传递位置参数，通过kwargs传递关键字参数

### 控制线程名字

In [20]:
threading.Thread(target=add, args=(1,2), name='add').start()

2017-04-14 15:30:47,298 INFO  [add] 3


通过name参数能控制线程名字

In [25]:
import time

In [26]:
def worker():
    logging.info('starting')
    time.sleep(2)
    logging.info('completed')

In [27]:
t1 = threading.Thread(target=worker, name='worker')
t2 = threading.Thread(target=worker, name='worker')

In [28]:
t1.start()
t2.start()

2017-04-14 15:50:18,040 INFO  [worker] starting
2017-04-14 15:50:18,041 INFO  [worker] starting
2017-04-14 15:50:20,045 INFO  [worker] completed
2017-04-14 15:50:20,048 INFO  [worker] completed


In [33]:
t1.start()       ##  再运行，报错 “线程只能运行一次”

RuntimeError: threads can only be started once

线程可以重名，线程名并不是线程的唯一标识，但是通常应该避免线程重名，通常的处理手段是加前缀

In [30]:
t1 == t2

False

In [31]:
t1.ident    ## 得到当前一个线程号

139822677071616

In [32]:
t2.ident

139822693857024

In [34]:
t1.ident == t2.ident

False

In [35]:
help(t1)

Help on Thread in module threading object:

class Thread(builtins.object)
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None)
 |      This constructor should always be called with keyword arguments. Arguments are:
 |      
 |      *group* should be None; reserved for future extension when a ThreadGroup
 |      class is implemented.
 |      
 |      *target* is the callable object to be invoked by the run()
 |      method. Defaults to None, meaning nothing is called.
 |      
 |      *name* is the thread name. By default, a unique name is constructed of
 |      the form "Thread-N" where N is a small decimal number.
 |      
 |      *args* is the argu

#### logging是常用的功能，自学！并不困难！

In [36]:
t1.is_alive()   ## 这是一个方法，加括号了 ，表示这个线程是否还活着

False

注：线程，不能保证它的执行顺序了！

In [37]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(processName)s [%(threadName)s] %(message)s')

In [38]:
logging.warning('haha')

2017-04-14 16:52:11,793 WARNING  [MainThread] haha


In [39]:
## 加了 %(processName)s  没有显示？ basicConfig后，要重新reload !!!

#### 建议在Linux或MAC环境下编程，Win环境不建议！

### daemon 与 non-daemon

In [40]:
t = threading.Thread(target=worker)

In [41]:
t.daemon      ## 有个 daemon 属性

False

In [47]:
t = threading.Thread(target=worker, daemon=True)

In [48]:
t.daemon

True

In [49]:
t.start()       ## 等于True的时候，观察与 non-daemon 没有任何区别

2017-04-14 17:11:56,491 INFO  [Thread-14] starting
2017-04-14 17:11:58,499 INFO  [Thread-14] completed


In [51]:
t = threading.Thread(target=worker)   ## 不配置时，默认是false的

In [52]:
t.start()         ##  没有任何区别！   看不出区别来

2017-04-14 17:13:30,666 INFO  [Thread-16] starting
2017-04-14 17:13:32,675 INFO  [Thread-16] completed


根据在pycharm中执行的结果总结：
线程退出时，其(标记为)daemon子线程也会退出，而non-daemon子线程不会(跟父线程)退出

那以上，为什么在ipython中non-daemon没有退出呢？因为ipython主线程并没有退出

In [2]:
import logging
import importlib
import time

importlib.reload(logging) 

<module 'logging' from '/root/.pyenv/versions/3.5.2/lib/python3.5/logging/__init__.py'>

In [3]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

In [4]:
def worker():
    logging.info('starting')
    time.sleep(2)
    logging.info('completed')

In [5]:
t = threading.Thread(target=worker)

In [6]:
t.daemon

False

In [7]:
t.start() 

2017-04-14 20:10:29,441 INFO  [Thread-4] starting
2017-04-14 20:10:31,454 INFO  [Thread-4] completed


In [8]:
threading.currentThread().is_alive()      ##  看，并没结束    如果在IDE里，就结束了

True

In [ ]:
##  pycharm里执行 

import logging
import importlib
import time
import threading

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

def worker():
    logging.info('starting')
    time.sleep(2)
    logging.info('completed')

if __name__ == "__main__":
    logging.info('starting-1')

    t1 = threading.Thread(target=worker, name='non-daemon')
    t1.start()
    t2 = threading.Thread(target=worker, name='daemon',daemon=True)
    t2.start()
    logging.info('completed-1')

In [ ]:
## 得到结果：
D:\Python\Python35\python.exe D:/py/lnh/test/log.py
2017-04-14 20:22:13,939 INFO  [MainThread] starting-1
2017-04-14 20:22:13,940 INFO  [non-daemon] starting
2017-04-14 20:22:13,940 INFO  [daemon] starting
2017-04-14 20:22:13,940 INFO  [MainThread] completed-1
2017-04-14 20:22:15,940 INFO  [non-daemon] completed

In [ ]:
## 也会出现这样的结果：
D:\Python\Python35\python.exe D:/py/lnh/test/log.py
2017-04-14 20:23:20,788 INFO  [MainThread] starting-1
2017-04-14 20:23:20,789 INFO  [non-daemon] starting
2017-04-14 20:23:20,789 INFO  [daemon] starting
2017-04-14 20:23:20,789 INFO  [MainThread] completed-1
2017-04-14 20:23:22,789 INFO  [daemon] completed
2017-04-14 20:23:22,789 INFO  [non-daemon] completed 
        
## 这所以这样，是因为comyn偷机取巧，假定t1每次都先完成！！！

这里为什么得不到想要的结果呢？
t1和t2分别注释执行，看看效果，就OK了！

In [ ]:
import logging
import importlib
import time
import threading

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

def worker():
    logging.info('starting')
    time.sleep(3)
    logging.info('completed')

if __name__ == "__main__":
    logging.info('starting-1')

    # t1 = threading.Thread(target=worker, name='non-daemon')
    # t1.start()
    t2 = threading.Thread(target=worker, name='daemon',daemon=True)
    t2.start()
    logging.info('completed-1')

In [ ]:
得到结论：(始终是这样了)

D:\Python\Python35\python.exe D:/py/lnh/test/log.py
2017-04-14 20:30:19,130 INFO  [MainThread] starting-1
2017-04-14 20:30:19,130 INFO  [daemon] starting
2017-04-14 20:30:19,130 INFO  [MainThread] completed-1

In [ ]:
开启 t1 注释了 t2 后 始终是：   (符合我们的预期)         

D:\Python\Python35\python.exe D:/py/lnh/test/log.py      ## 这块好像贴错啦
2017-04-14 20:30:19,130 INFO  [MainThread] starting-1
2017-04-14 20:30:19,130 INFO  [daemon] starting          ## 应该贴出来是non-daemon的
2017-04-14 20:30:19,130 INFO  [MainThread] completed-1
        
        
pycharm重新运行后贴上：

D:\Python\Python35\python.exe D:/py/lnh/test/log.py
2017-04-17 15:38:56,983 INFO  [MainThread] starting-1
2017-04-17 15:38:56,983 INFO  [non-daemon] starting
2017-04-17 15:38:57,984 INFO  [MainThread] completed-1
2017-04-17 15:38:59,984 INFO  [non-daemon] completed

** 线程退出时，其(标记为)daemon子线程也会退出，而non-daemon子线程不会(跟父线程)退出 **

In [ ]:
这样就OK啦！！！！！

import logging
import importlib
import time
import threading

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s  [%(threadName)s] %(message)s')

def worker():
    logging.info('starting')
    time.sleep(3)
    logging.info('completed')

if __name__ == "__main__":
    logging.info('starting-1')

    t1 = threading.Thread(target=worker, name='non-daemon')
    t1.start()

    time.sleep(1)           ## 中间睡 1 秒

    t2 = threading.Thread(target=worker, name='daemon',daemon=True)
    t2.start()
    logging.info('completed-1')

In [ ]:
始终是这样啦：

D:\Python\Python35\python.exe D:/py/lnh/test/log.py
2017-04-14 20:34:55,059 INFO  [MainThread] starting-1
2017-04-14 20:34:55,059 INFO  [non-daemon] starting
2017-04-14 20:34:56,059 INFO  [daemon] starting
2017-04-14 20:34:56,059 INFO  [MainThread] completed-1
2017-04-14 20:34:58,059 INFO  [non-daemon] completed

** 线程的退出会等待所有的non-daemon子线程退出；所有的non-daemon退出后，线程才会退出**

总结： 线程退出时，其(标记为)daemon子线程也会退出，而non-daemon子线程不会(跟父线程)退出

上面这两句是不是冲突？！MD！父线程并不是主线程的概念！

补充：join方法会阻塞直到线程退出或者超时，timeout 是可选的，如果不设置timeout,会一直等待线程退出！ join方法用的也是比较多的！

谁调用join，就阻塞谁！   join补充在下一课！

注：线程不能kill，一旦启动，只能等待自己退出！

线程A启动了线程B，那么A是B的父线程，如果B.daemon == True    A退出，会杀死B
                                    如果B.daemon == False   A退出，会等待B 

** 终于理解啦 **

In [ ]:
non-daemon子线程是不会（跟父线程）退出，但是父线程会等待它，再退出！

继续：

In [9]:
threading.enumerate()        ## 它会获取当前所有的线程，是一个数组

[<_MainThread(MainThread, started 140655429027648)>,
 <Thread(Thread-2, started daemon 140655078668032)>,
 <Heartbeat(Thread-3, started daemon 140655070275328)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140655045097216)>,
 <ParentPollerUnix(Thread-1, started daemon 140655036704512)>]

In [10]:
for t in threading.enumerate():
    print(t.name)

MainThread
Thread-2
Thread-3
IPythonHistorySavingThread
Thread-1


之前创建线程都是通过实例化Thread类

In [11]:
class MyThread(threading.Thread):     ##  通过继承的方式    
    def run(self):
        logging.info('run')

In [12]:
t = MyThread()

In [13]:
t.start()

2017-04-15 07:02:14,176 INFO  [Thread-5] run


但是Python中通常不使用这种方法！只是告诉大家可以使用这种方法。

In [14]:
t.run()      ## 通过继承这种方式过来的，run方法可以使用   

2017-04-15 07:04:36,467 INFO  [MainThread] run


区别：run方法是在主线程里面执行，start方法是在线程里面执行的。

In [15]:
t = threading.Thread(target=worker)    ## 当以这种方式初始化

In [16]:
t.run()    ##  它也可以执行，不过是在主线程里执行的！

2017-04-15 07:07:32,796 INFO  [MainThread] starting
2017-04-15 07:07:34,823 INFO  [MainThread] completed


In [18]:
t = threading.Thread(target=worker) 

In [19]:
t.start()

2017-04-15 07:09:03,986 INFO  [Thread-7] starting
2017-04-15 07:09:06,001 INFO  [Thread-7] completed


注：一旦执行了start方法，run方法就不能执行了；反之亦然！


如果不是以继承的方式创建线程，run方法和start方法只能执行其中一个。

通常不建议使用继承的方式！

为什么只能执行一次呢？引入新的概念

**thread local**

这个暂时了解就好

In [2]:
import threading

In [3]:
threading.local()      

In [21]:
ctx = threading.local()       ## 实例化吧

In [22]:
ctx.data = 5   ## 增加一个属性

In [23]:
ctx.data

5

In [24]:
def worker():
    logging.info(ctx.data)

In [25]:
worker()

2017-04-15 07:14:38,085 INFO  [MainThread] 5


In [26]:
## 为了对比 ，再增加一个变量

data = 'abc'

In [27]:
def worker():
    logging.info(data)
    logging.info(ctx.data)

In [28]:
worker()             ## 当在一个主线程里，是OK的

2017-04-15 07:17:29,860 INFO  [MainThread] abc
2017-04-15 07:17:29,867 INFO  [MainThread] 5


In [29]:
threading.Thread(target=worker).start()       ## 执行后，打印了abc，但是没有data属性

2017-04-15 07:18:59,200 INFO  [Thread-8] abc
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/root/.pyenv/versions/3.5.2/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/root/.pyenv/versions/3.5.2/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-27-c46dda5d9c3f>", line 3, in worker
    logging.info(ctx.data)
AttributeError: '_thread._local' object has no attribute 'data'



In [ ]:
## 执行后，打印了abc，但是没有data属性 ；这是因为什么呢？

thread local 对象定义的属性，只在当前线程可见！

ctx.data是(ctx)对象的属性，data则是一个变量；觉得应该是这样理解的。

In [ ]:
## 想想 thread local 有什么用？
线程独享资源

thread local 暂时了解就好！

继续：

### 定时器/延迟执行

In [30]:
help(threading.Timer)

Help on class Timer in module threading:

class Timer(Thread)
 |  Call a function after a specified number of seconds:
 |  
 |  t = Timer(30.0, f, args=None, kwargs=None)
 |  t.start()
 |  t.cancel()     # stop the timer's action if it's still waiting
 |  
 |  Method resolution order:
 |      Timer
 |      Thread
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, interval, function, args=None, kwargs=None)
 |      This constructor should always be called with keyword arguments. Arguments are:
 |      
 |      *group* should be None; reserved for future extension when a ThreadGroup
 |      class is implemented.
 |      
 |      *target* is the callable object to be invoked by the run()
 |      method. Defaults to None, meaning nothing is called.
 |      
 |      *name* is the thread name. By default, a unique name is constructed of
 |      the form "Thread-N" where N is a small decimal number.
 |      
 |      *args* is the argument tuple for the target invoc

In [32]:
def worker():
    logging.info('run')

In [33]:
t = threading.Timer(interval=5, function=worker)

In [34]:
t.start()    ## 并没有直接执行，大约5秒后执行       就是延迟执行

2017-04-15 08:53:28,712 INFO  [Thread-10] run


In [35]:
t.is_alive()

False

In [36]:
t.start()

RuntimeError: threads can only be started once

In [37]:
t = threading.Timer(interval=5, function=worker)

In [38]:
t.name = 'worker'

In [39]:
t.daemon   ## 默认也是false

False

In [40]:
t.daemon = True

In [41]:
t.start()          ## ok ，name 设置成功了

2017-04-15 08:56:38,597 INFO  [worker] run


In [42]:
t = threading.Timer(interval=5, function=worker)

In [43]:
t.start()

In [44]:
t.cancel()    ## 5秒内运行，结束了它

In [45]:
threading.enumerate()       ## 事实上已经不存在了

[<_MainThread(MainThread, started 140655429027648)>,
 <Thread(Thread-2, started daemon 140655078668032)>,
 <Heartbeat(Thread-3, started daemon 140655070275328)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140655045097216)>,
 <ParentPollerUnix(Thread-1, started daemon 140655036704512)>]

In [ ]:
##  所以，Timer可以延迟执行，并且可以被 cancel 

问题来了，如何利用Timer实现一个循环执行？

In [53]:
t = threading.Timer(interval=300, function=worker)

In [54]:
t.name='worker'

In [55]:
t.start()

In [56]:
t.cancel()

In [57]:
threading.enumerate()         ## 并没有它的线程

[<_MainThread(MainThread, started 140655429027648)>,
 <Thread(Thread-2, started daemon 140655078668032)>,
 <Heartbeat(Thread-3, started daemon 140655070275328)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140655045097216)>,
 <ParentPollerUnix(Thread-1, started daemon 140655036704512)>]

In [58]:
for x in threading.enumerate():
    print(x.name)               ## 并没有 worker 这个线程，所以cancel后，就结束了！

MainThread
Thread-2
Thread-3
IPythonHistorySavingThread
Thread-1


In [59]:
t = threading.Timer(interval=300, function=worker)

In [60]:
t.name = 'worker'

In [61]:
t.start()

In [62]:
threading.enumerate()           ## 没使用cancel前，第4行就是 worker 线程

[<_MainThread(MainThread, started 140655429027648)>,
 <Thread(Thread-2, started daemon 140655078668032)>,
 <Heartbeat(Thread-3, started daemon 140655070275328)>,
 <Timer(worker, started 140655028311808)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140655045097216)>,
 <ParentPollerUnix(Thread-1, started daemon 140655036704512)>]

In [63]:
t.cancel()

In [64]:
threading.enumerate()      ## 没了

[<_MainThread(MainThread, started 140655429027648)>,
 <Thread(Thread-2, started daemon 140655078668032)>,
 <Heartbeat(Thread-3, started daemon 140655070275328)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140655045097216)>,
 <ParentPollerUnix(Thread-1, started daemon 140655036704512)>]

In [66]:
t.is_alive()

False

注：普通线程是不可以被cancel的！

In [70]:
def worker():
    logging.info('starting')
    time.sleep(30)
    logging.info('completed')

In [71]:
t = threading.Timer(interval=0, function=worker)      ## 这里延迟执行0秒，说明没有被延迟

In [72]:
t.start()

2017-04-15 09:12:50,919 INFO  [Thread-18] starting


In [73]:
t.cancel()

In [74]:
t.is_alive()          ##  它并没有被 cancel 掉      ## 30秒后，执行出以下

True

2017-04-15 09:13:20,954 INFO  [Thread-18] completed


In [ ]:
##  它并没有被 cancel 掉  ,为什么呢？

当function参数所指定函数开始执行的时候，cancel无效！只有在执行之前，才有效！

推荐APSchedule这个第三方库，是强大的定时执行的功能！

Timer并不强大，实现不了晚上12点执行，这样的功能！